In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.keras.layers as tfl

2022-12-06 12:46:58.426685: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.models import Model
import tensorflow.keras.optimizers as optimizers
import tensorflow.keras.metrics as metrics

In [3]:

x_train = np.load("./cleaned_data/small/x_train.npy")
y_train = np.load("./cleaned_data/small/y_train.npy")
train_mu = np.load("./cleaned_data/small/train_mu.npy")
train_std = np.load("./cleaned_data/small/train_std.npy")
x_val = np.load("./cleaned_data/small/x_val.npy")
y_val= np.load("./cleaned_data/small/y_val.npy")


In [4]:
# y = [no sack, sack, time to sack]
# x = (M, 23, 11)
x_train.shape

(5991, 23, 11)

In [26]:
import keras.backend as K

def my_loss(y_true, y_output):
    bce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    mse = tf.keras.losses.MeanSquaredError()
    
    # get mse of only true positives
    true_sack_mask = y_true[:1]==1
    # checked, mse function can take in array of length zero
    return bce(y_true[:,0:-1], y_output[:,0:-1]) + mse(y_true[true_sack_mask], y_output[true_sack_mask])

def bce_metric(y_true, y_output):
    return K.mean(K.binary_crossentropy(y_true[:,0:-1], y_output[:,0:-1], from_logits=False))

def mse_metric(y_true, y_output):
    # get mse of only true positives
    true_sack_mask = y_true[:1]==1
    return K.mean(K.square(y_true[true_sack_mask] - y_output[true_sack_mask]), axis=-1)
    # return K.mean(K.square(y_pred[:,-1] - y_true[:,-1]), axis=-1)
    
def accuracy_metric(y_true, y_output):
    preds = K.cast(K.argmax(y_output[:,0:-1], axis=-1), 'float32')
    return K.mean(K.cast(y_true[:,1] == preds, 'float32'))

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def recall(y_true, y_output):
    preds = K.cast(K.argmax(y_output[:,0:-1], axis=-1), 'float32')
    true_positives = K.sum(K.round(K.clip(y_true[:,1] * preds, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true[:,1], 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_output):
    preds = K.cast(K.argmax(y_output[:,0:-1], axis=-1), 'float32')
    true_positives = K.sum(K.round(K.clip(y_true[:,1] * preds, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(preds, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


In [27]:
''' 
Model

how cropping layer works - https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-cropping-layers-with-keras.md
'''

' \nModel\n\nhow cropping layer works - https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-cropping-layers-with-keras.md\n'

In [28]:
''' 
Output:
[prob of no sack, prob of sack, time till sack]

If predict no sack, doesn't matter what time is (recorded as -1 in training data)
'''

def createModel(input_shape = (23,7)):
    
    X = tfl.Input(input_shape)  # define the input to the model
    flat = tfl.Flatten(input_shape=(23, 7))(X)     # Flatten to pass into linear layers
    d1 = tfl.Dense(50, activation='relu')(flat)
    d3 = tfl.Dense(3,activation=None)(d1)
    
    # have layer (batch_size, 3). Want to take (b, [0,1]) and turn them into probabilities, and keep (b, [2]) as time
    # https://datascience.stackexchange.com/questions/86740/how-to-slice-an-input-in-keras
    intermediate = tfl.Reshape((3,1), input_shape=(3,))(d3)
    
    probs = tfl.Cropping1D(cropping=(0,1))(intermediate)
    probs = tfl.Reshape((2,), input_shape=(2,1))(probs)
    probs = tfl.Activation('softmax')(probs)
    
    time = tfl.Cropping1D(cropping=(2,0))(intermediate)
    time = tfl.Reshape((1,), input_shape=(1,1))(time)
    
    # concatenate the probabilities and predicted_time_to_sack back into one layer
    out = tfl.Concatenate(axis=-1)([probs, time])
    
    model = Model(inputs=X, outputs=out)        # create model
    
    return model
    

In [29]:
model = createModel()

print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 23, 7)]      0           []                               
                                                                                                  
 flatten_3 (Flatten)            (None, 161)          0           ['input_4[0][0]']                
                                                                                                  
 dense_6 (Dense)                (None, 50)           8100        ['flatten_3[0][0]']              
                                                                                                  
 dense_7 (Dense)                (None, 3)            153         ['dense_6[0][0]']                
                                                                                            

In [30]:
LEARNING_RATE = 0.001
BETA_1 = 0.9
BETA_2 = 0.999
EPS = 1e-07

opt = optimizers.Adam(
    learning_rate=LEARNING_RATE,
    beta_1=BETA_1,
    beta_2=BETA_2,
    epsilon=EPS)

model.compile(loss = my_loss, optimizer = opt, metrics = [accuracy_metric, bce_metric, mse_metric, recall, precision])

In [31]:
# Better optimizer

lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=LEARNING_RATE,
    decay_steps=10000,
    decay_rate=0.9)

scheduled_opt = optimizers.Adam(
    learning_rate=lr_schedule,
    beta_1=BETA_1,
    beta_2=BETA_2,
    epsilon=EPS)

In [32]:
NUM_EPOCHS = 10
history = model.fit(x_train[:,:,4:], y_train, epochs=NUM_EPOCHS)

Epoch 1/10
188/188 [==============================] - 2s 2ms/step - loss: 0.3280 - accuracy_metric: 0.9403 - bce_metric: 0.2534 - mse_metric: 0.0735 - recall: 0.0000e+00 - precision: 0.0000e+00
Epoch 2/10
188/188 [==============================] - 0s 2ms/step - loss: 0.2730 - accuracy_metric: 0.9415 - bce_metric: 0.2224 - mse_metric: 0.0496 - recall: 0.0000e+00 - precision: 0.0000e+00
Epoch 3/10
188/188 [==============================] - 0s 2ms/step - loss: 0.2247 - accuracy_metric: 0.9415 - bce_metric: 0.2106 - mse_metric: 0.0134 - recall: 0.0027 - precision: 0.0053
Epoch 4/10
188/188 [==============================] - 0s 2ms/step - loss: 0.2580 - accuracy_metric: 0.9409 - bce_metric: 0.2051 - mse_metric: 0.0532 - recall: 0.0013 - precision: 0.0053
Epoch 5/10
188/188 [==============================] - 1s 3ms/step - loss: 0.2576 - accuracy_metric: 0.9425 - bce_metric: 0.1934 - mse_metric: 0.0635 - recall: 0.0130 - precision: 0.0319
Epoch 6/10
188/188 [==============================] - 

In [33]:
train_no_sack = np.round(np.sum(y_train[:,0])/len(y_train), 3)
print(f"Percentage of training plays with no sack = {train_no_sack}")

Percentage of training plays with no sack = 0.941


In [34]:
model.predict(x_train[0:5,:,4:])

1/1 [==============================] - 0s 155ms/step


array([[ 0.9956393 ,  0.00436067,  0.5960429 ],
       [ 0.9466307 ,  0.05336923,  0.24010435],
       [ 0.9317543 ,  0.06824572, -0.2516085 ],
       [ 0.82877773,  0.1712223 ,  1.6426669 ],
       [ 0.87881196,  0.12118809,  1.266871  ]], dtype=float32)

In [35]:
metrics_df = pd.DataFrame(history.history)


In [36]:
metrics_df.tail()

,loss,accuracy_metric,bce_metric,mse_metric,recall,precision
5,0.230476,0.942653,0.188273,0.041505,0.031472,0.051418
6,0.212052,0.944481,0.174995,0.036393,0.046277,0.111702
7,0.229667,0.945479,0.168753,0.060211,0.066076,0.127660
8,0.256913,0.945882,0.161015,0.097150,0.092553,0.172872
9,0.224968,0.948874,0.151122,0.073586,0.119681,0.247340


In [37]:
num_no_sack = np.round(np.sum(y_val[:,0])/len(y_val), 3)
print(f"Percentage of validation plays with no sack = {num_no_sack}")

Percentage of validation plays with no sack = 0.947


In [38]:
val_loss, cat_acc, val_bce, val_mse, val_recall, val_precision = model.evaluate(x_val[:,:,4:], y_val, verbose=2)

41/41 - 1s - loss: 0.3561 - accuracy_metric: 0.9405 - bce_metric: 0.2590 - mse_metric: 0.0896 - recall: 0.0110 - precision: 0.0366 - 633ms/epoch - 15ms/step


In [39]:

print(f"val loss = {val_loss}")
print(f"categorical accuracy = {cat_acc}")
print(f"val_bce = {val_bce}")
print(f"val_mse = {val_mse}")
print(f"val_recall = {val_recall}")
print(f"val_precision = {val_precision}")

val loss = 0.35607048869132996
categorical accuracy = 0.9405487775802612
val_bce = 0.25898969173431396
val_mse = 0.08964574337005615
val_recall = 0.01097560953348875
val_precision = 0.03658536449074745


In [40]:
model.predict(x_val[:,:,4:])[:,0]

41/41 [==============================] - 0s 2ms/step


array([0.8872072 , 0.9867906 , 0.8555738 , ..., 0.99581367, 0.9926259 ,
       0.99644214], dtype=float32)

In [37]:
# https://www.kdnuggets.com/2021/02/saving-loading-models-tensorflow.html

model_string = f"models/third_model/weights_epochs{NUM_EPOCHS}"
model.save_weights(model_string)